# FINAL PROJECT: FUNCTIONAL PROGRAMMING E-COMMERCE SYSTEM

### Isabel de Paula Diez de Rivera Vergara - NIA : 100429935@alumnos.uc3m.es
### Ivan Ocheretianyi - NIA: 100487317@alumnos.uc3m.es

# Introduction

This project builds a small e-commerce system in Haskell. It models the main parts of an online shop using well-defined types: **Category** and **Product** describe what the shop sells, **Customer** and **LoyaltyLevel** show who is buying and which discounts apply, **CartItem** and **ShoppingCart** represent what the customer wants to buy, **Stock** stores available quantities, and **Order** combines all of this with a **Status**.

Discounts are handled through the *Discountable* type class. Both **LoyaltyLevel** and **Category** are instances of this class, so the final price of each item is obtained by applying both discounts in order. The function `calculateOrderTotal` then adds everything together.

The system also checks correctness. Equality for **Product** and **Customer** is based on their identifiers, stock is verified through `stockQuantity` and `checkStock`, and `createOrder` ensures that orders with missing items are rejected.

Order progress is controlled by the **Status** type. The function `transitionAllowed` defines which changes are valid, and `updateOrderStatus` returns a new order only when the transition is permitted, using `Maybe` to represent success or failure.

The project also includes a small search module. Orders can be filtered using different criteria such as customer id, loyalty level, product id, category, or total price. High-value customers are identified using helper functions like `activeOrders`, `customersOf`, `uniq`, and `totalOfCustomer`.

Finally, these pure functions are meant to be combined with I/O so users can search products, manage carts, place orders, and so that shop owners can inspect customers and process pending orders.

# CODE

## 1. Exhaustive Type Definition: 
Define exhaustively all the former concepts, using the most appropriate way (type, data, newtype) for each of them. Ensure that aliases are defined as needed to make arguments of the new types easier to understand.
## 2. Pretty Printing Functions: 
Functions to show the former concepts in a clear and readable way by implementing the Show Type Class.

+ **Category** is a custom `data` type that represents the type of product sold in the shop. It includes four fixed options: `Electronics`, `Books`, `Clothing`, and `Groceries`. The type uses `deriving (Show, Eq, Read)` so that values can be printed, compared or read from file automatically.
<br><br>
+ **Product** is a custom `data` type that stores information about each item in the shop. It has four fields:
  + `pid` of type `Int` for the product’s unique id,
  + `pname` of type `String` for the product name,
  + `price` of type `Float`,
  + and `category` of type **Category**.
  + A custom `Show` instance is defined so that products are printed in a clear and readable way with `totalPrice` printed with 2 decimal number. 
  + Additionally, `Eq` instance is defined to check equality of products by their `pid`s
<br><br>
+ **LoyaltyLevel** is a `data` type describing the customer’s loyalty status: `Bronze`, `Silver`, or `Gold`. It uses `deriving (Show, Eq, Read)` to allow printing, equality checks and read from file.
<br><br>
+ **Customer** is a custom `data` type storing information about a client. It contains:
  + `cid` of type `Int` as the customer id,
  + `cname` of type `String` for the customer’s name,
  + and a loyalty level of type **LoyaltyLevel**.
  + A custom `Show` instance prints the customer nicely,
  + while `Eq` instance allows comparison of equality
<br><br>
+ **CartItem** is a custom `data` type representing one line inside a shopping cart. It has two fields:
  + `product` of type **Product**
  + and `quantity` of type `Int`.
  + Its custom `Show` instance prints each item together with the number of units
  + as well as `Eq` compares **CartItem** by `product` and `quantity`.
<br><br>
+ **ShoppingCart** is a `newtype` wrapper around a list of **CartItem** values. The wrapper makes the type safer and easier to handle. 
  + The custom `Show` instance prints the whole cart item by item, using indentation for clarity.
  + The custom `Eq` instance compares two **ShoppingCart**s by there content.
<br><br>
+ **Stock** is a `newtype` wrapper around a list of pairs `(Product, Int)`. Each pair stores a product and the number of units available. Its `Show` instance prints the whole stock clearly, showing product info and available quantity.
<br><br>
+ **Status** is a `data` type that represents the state of an order. It has five possible values: `Pending`, `Processing`, `Shipped`, `Delivered`, and `Cancelled`. This type allows the program to model order progression. As well as **LoyaltyLevel** it is an instance of `Show, Eq, Read` classes.
<br><br>
+ **Order** is a custom `data` type representing a complete purchase. It contains:
  + a **Customer**,
  + a **ShoppingCart**,
  + a `totalPrice` of type `Float`,
  and a **Status**.
  + Its `Show` instance prints all this information in a readable format with `totalPrice` printed with 2 decimal number.
  + Its `Eq` instance compares orders by **Customer** and **ShoppingCart**.
<br><br>
+ **SearchCriterion** is a `data` type used to filter lists of orders. It includes several constructors: `ById`, `ByLoyaltyLevel`, `ByProductId`, `ByCategory`, and `ByTotalPrice`, each carrying the needed value.
<br><br>
+ **Discountable** is a type class that defines the function `applyDiscount`. Any type that becomes an instance of this class must explain how it transforms a price.
  Both **LoyaltyLevel** and **Category** are instances, meaning the program can apply discounts based on customer loyalty, product category, or a combination of both.

In [1]:
import System.IO  -- first load modules
import Data.Char (isDigit, toLower)
import Text.Read (readMaybe)
import Data.Maybe (fromMaybe)
import Text.Printf (printf)

In [2]:
data Category = Electronics | Books | Clothing | Groceries deriving (Show, Eq, Read)

data Product = Product {pid :: Int, pname :: String, price :: Float, category :: Category}
instance Show Product where
    show (Product id name price category) = "Product id: " ++ show id ++ ", product: " ++ show name ++ ", price: " ++ printf "%.2f" price ++ ", category: " ++ show category

instance Eq Product where
    (Product pid1 _ _ _) == (Product pid2 _ _ _) = pid1 == pid2

In [3]:
data LoyaltyLevel = Bronze | Silver | Gold deriving (Show, Eq, Read)

data Customer = Customer {cid :: Int, cname :: String, loyaltyLevel :: LoyaltyLevel}

instance Show Customer where
    show (Customer id name loyaltyLevel) = "Customer id: " ++ show id ++ ", customer: " ++ show name ++ ", loyalty level: " ++ show loyaltyLevel

instance Eq Customer where
  (Customer id1 _ _) == (Customer id2 _ _) = id1 == id2

In [4]:
data CartItem = CartItem {product :: Product, quantity :: Int}

instance Show CartItem where
    show (CartItem product quantity) = "Item: " ++ show product ++ ", units: " ++ show quantity

instance Eq CartItem where
        (CartItem p1 q1) == (CartItem p2 q2) = p1 == p2 && q1 == q2

In [5]:
newtype ShoppingCart = ShoppingCart [CartItem]

instance Show ShoppingCart where
    show (ShoppingCart items) = "Shopping cart:\n" ++ unlines (map (\x -> '\t': show x) items)

instance Eq ShoppingCart where
        (ShoppingCart items1) == (ShoppingCart items2) = items1 == items2

In [6]:
newtype Stock = Stock [(Product, Int)]

instance Show Stock where
    show (Stock items) = "Stock:\n" ++ unlines (map (\(p, i) -> "\t" ++ show p ++ ", quantity available: " ++ show i) items)

data Status = Pending | Processing | Shipped | Delivered | Cancelled deriving (Show, Read, Eq)

In [7]:
data Order = Order {customer :: Customer, shoppingCart :: ShoppingCart, totalPrice :: Float, status :: Status}

instance Show Order where
    show (Order customer shoppingCart totalPrice status) = "Order for customer: " ++ show customer ++ "\n" ++ show shoppingCart 
        ++ "Total price: " ++ printf "%.2f" totalPrice ++ ", status: " ++ show status ++ "\n"

instance Eq Order where
    (Order (Customer cid1 _ _) shpcrt1 _ _) == (Order (Customer cid2 _ _) shpcrt2 _ _) = cid1 == cid2 && shpcrt1 == shpcrt2

data SearchCriterion = ById Int | ByLoyaltyLevel LoyaltyLevel | ByProductId Int | ByCategory Category | ByTotalPrice Float deriving Show

In [8]:
class Discountable a where
  applyDiscount :: a -> Float -> Float

instance Discountable LoyaltyLevel where
  applyDiscount Bronze p = p
  applyDiscount Silver p = p * 0.95
  applyDiscount Gold   p = p * 0.90

instance Discountable Category where
  applyDiscount Books p = p * 0.85 -- Books 15% off
  applyDiscount _     p = p

**Error** is a `newtype` wrapper around a list of **Product**s. It is used in function 7 and represent the list of products that are out of stock. Its derives `Show` letting print all the missing products

In [9]:
newtype Error = Error [Product] --created due to 7
instance Show Error where
    show (Error products) = "Error: The following products are out of stock:\n" ++ unlines (map (\p -> '\t': show p) products)

### Testing

We ran these small tests to verify that each of our types is correctly constructed and printed. By creating example values such as **Product**, **CartItem**, **ShoppingCart**, **Stock**, **Customer**, and **Order**, we checked that their fields are stored properly and that our custom `Show` instances display the information in a clear and readable way. This confirms that the basic data structures of the system behave as expected.

In [10]:
product1 = Product 1 "Charger type-C" 12 Electronics
product2 = Product 2 "T-shirt" 15 Clothing
product3 = Product 3 "Watermelon" 5 Groceries

cart1 = CartItem product1 2
cart2 = CartItem product2 5

cart = ShoppingCart [cart1,  cart2]
cart

stock = Stock (zip [product1, product2, product3] [2, 3, 4])

stock

customer = Customer 1 "Ivan" Gold
order = Order customer cart 89.1 Pending

order

error = Error [product1, product2]
error

Shopping cart:
	Item: Product id: 1, product: "Charger type-C", price: 12.00, category: Electronics, units: 2
	Item: Product id: 2, product: "T-shirt", price: 15.00, category: Clothing, units: 5

Stock:
	Product id: 1, product: "Charger type-C", price: 12.00, category: Electronics, quantity available: 2
	Product id: 2, product: "T-shirt", price: 15.00, category: Clothing, quantity available: 3
	Product id: 3, product: "Watermelon", price: 5.00, category: Groceries, quantity available: 4

Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Charger type-C", price: 12.00, category: Electronics, units: 2
	Item: Product id: 2, product: "T-shirt", price: 15.00, category: Clothing, units: 5
Total price: 89.10, status: Pending

Error: The following products are out of stock:
	Product id: 1, product: "Charger type-C", price: 12.00, category: Electronics
	Product id: 2, product: "T-shirt", price: 15.00, category: Clothing

## 2. Pretty printing

All test outputs are technically correct but some of them are hard to read in Jupyter, since lists and strings were shown on a single line with escaped characters. To improve human readability without changing program logic, we introduced two helpers:
haskell.

These following functions let us display and format lists or stored data more clearly during testing. They are an addition to the created custom **Show** instances.

These additions do not affect the computation; they simply make test results easier to interpret.

* `splitOn` splits **String** into a list of **String** by a given `delimiter`

* `listToString` is a pretty-printer for list with each element on a new line

In [11]:
splitOn :: Char -> String -> [String]
splitOn _ "" = [""]
splitOn delimiter (x:xs)
    | x == delimiter = "" : rest
    | otherwise = (x : head rest) : tail rest
  where
    rest = splitOn delimiter xs
    
listToString :: Show a => [a] -> String
listToString = foldr (\x acc -> show x ++ "\n" ++ acc) ""

## 3. Price Calculation

The function `calculateProductPrice` was designed to extract only the price field from a **Product**, ignoring all other information via pattern matching: we match the constructor, discard the unused fields, and return the price `p`. This keeps the function efficient.

In [12]:
calculateProductPrice :: Product -> Float
calculateProductPrice (Product _ _ p _) = p

### Testing

In [13]:
product1 = Product 1 "Charger type-C" 12 Electronics
product2 = Product 2 "T-shirt" 15 Clothing
product3 = Product 3 "Watermelon" 5 Groceries

calculateProductPrice product1
calculateProductPrice product2
calculateProductPrice product3

12.0

15.0

5.0

## 4. Discount Application

`myRound2` ensure prices display **exactly two decimals**. Floating-point results often keep many digits, but real-world monetary values cannot. This guarantees that every final price is realistic and displayed to two decimals.

`calculateOrderTotal` was designed to compute the final cost of an order by combining quantities and discounts in a clear way. For each `CartItem` in the `ShoppingCart`, we multiply the quantity by the discounted price of its `Product`. The helper `priceAfter` first applies the category discount and then the loyalty discount, using the `Discountable` type class. We use a list comprehension and `sum` to add all these subtotals, so the function directly reflects:

$$
\text{total} = \sum_{\text{items}} (\text{quantity} \times \text{discounted price})
$$

Before returning the result, we apply the helper `myRound2` to ensure the final price is expressed with exactly two decimals, representing realistic monetary values.

In [14]:
myRound2 :: Float -> Float
myRound2 x = fromIntegral rounded / 100
  where
    scaled     = x * 100
    integer    = floor scaled
    fraction   = scaled - fromIntegral integer
    rounded
      | fraction >= 0.5 = integer + 1
      | otherwise       = integer
    
calculateOrderTotal :: Order -> Float
calculateOrderTotal (Order cust (ShoppingCart items) _ _) =
  myRound2 (sum [ fromIntegral (quantity it) * priceAfter (product it) | it <- items ])
  where
    priceAfter prod = applyDiscount (loyaltyLevel cust) (applyDiscount (category prod) (calculateProductPrice prod))

### Testing

Verification test that `calculateOrderTotal` correctly applies the different loyalty discounts (0%, 5%, 10%) so that the same cart produces decreasing totals for Bronze, Silver, and Gold customers.

In [15]:
product1 = Product 1 "Ipad" 12 Electronics
product2 = Product 2 "Blouse"  15 Clothing
product3 = Product 3 "Lemons"  5 Groceries

cart1 = CartItem product1 2
cart2 = CartItem product2 5
cart  = ShoppingCart [cart1, cart2]

stock = Stock (zip [product1, product2, product3] [2, 3, 4])

customerGold   = Customer 1 "Ivan"   Gold
customerSilver = Customer 2 "Isa"  Silver
customerBronze = Customer 3 "Siro"  Bronze

orderGold   = Order customerGold   cart 0 Pending
orderSilver = Order customerSilver cart 0 Pending
orderBronze = Order customerBronze cart 0 Pending

calculateOrderTotal orderGold
calculateOrderTotal orderSilver
calculateOrderTotal orderBronze

89.1

94.05

99.0

To confirm that book purchases receive the 15% category discount **before** the loyalty discount, and that different loyalty levels correctly produce different final totals for the same book order.

In [16]:
productBook = Product 4 "Functional Programming for Dummies 101" 40 Books
cartBook1   = CartItem productBook 1
cartBook2   = CartItem productBook 2

cartwBook1 = ShoppingCart [cartBook1]
cartwBook2 = ShoppingCart [cartBook2]

orderBookGold1   = Order customerGold   cartwBook1 0 Pending
orderBookSilver1 = Order customerSilver cartwBook1 0 Pending
orderBookBronze1 = Order customerBronze cartwBook1 0 Pending

calculateOrderTotal orderBookGold1
calculateOrderTotal orderBookSilver1
calculateOrderTotal orderBookBronze1

30.6

32.3

34.0

To check that the system correctly handles a cart with different products by applying category and loyalty discounts only where appropriate and summing all item subtotals into one final total.

In [17]:
-- original cart + 2 books
cartMixed = ShoppingCart [cart1, cart2, cartBook2]
orderMixedGold = Order customerGold cartMixed 0 Pending

calculateOrderTotal orderMixedGold

150.3

## 5. Purchase

`addToCart` was designed to update a shopping cart in a simple and safe way. If the cart is empty, it just creates a new cart with the given `CartItem`. If not, it goes through the list of items with pattern matching: when it finds the same `Product` (using the `Eq` instance for **Product**), it increases its quantity; otherwise, it keeps the current item and continues recursively with the rest of the cart. The helper `add` is used to rebuild the cart when the product is not at the head, so we keep all existing items and only change the one that matches.

In [18]:
add :: CartItem -> ShoppingCart -> ShoppingCart
add item (ShoppingCart xs) = ShoppingCart (item:xs)

addToCart :: CartItem -> ShoppingCart -> ShoppingCart
addToCart item (ShoppingCart []) = ShoppingCart [item]
addToCart i@(CartItem product quantity) (ShoppingCart ((CartItem p q): cartList)) 
                | product == p = ShoppingCart (CartItem p (q+quantity) : cartList)
                | otherwise = add (CartItem p q) (addToCart i (ShoppingCart cartList))

### Testing

In [19]:
product1 = Product 1 "Ipad" 12 Electronics
product2 = Product 2 "Blouse"  15 Clothing
product3 = Product 3 "Lemons"  5 Groceries

cart1 = CartItem product1 2
cart2 = CartItem product2 5
cart3 = CartItem product3 7
cart4 = CartItem product2 3
cart  = ShoppingCart [cart1, cart2]
cart

Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.00, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.00, category: Clothing, units: 5

Add new product **Lemons**

In [20]:
cartWithLemons = addToCart cart3 cart
cartWithLemons

Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.00, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.00, category: Clothing, units: 5
	Item: Product id: 3, product: "Lemons", price: 5.00, category: Groceries, units: 7

Increase quantity of existing in cart product **Blouse**

In [21]:
cartWithMoreBlouses = addToCart cart4 cartWithLemons  -- increase quantity of existing product
cartWithMoreBlouses

Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.00, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.00, category: Clothing, units: 8
	Item: Product id: 3, product: "Lemons", price: 5.00, category: Groceries, units: 7

## 6. Cart Validation

`stockQuantity` was designed to search the stock list for a specific **Product** and return its available quantity. It uses list comprehension to collect all matching quantities, and if none are found it returns `0`.

`checkStock` uses `stockQuantity` to validate the cart. It scans all `CartItem` values in the **ShoppingCart**, and collects those products for which the available stock is smaller than the quantity requested. This gives a direct and readable way to detect missing or insufficient items before creating an order.

In [22]:
stockQuantity :: Stock -> Product -> Int
stockQuantity (Stock xs) p
  | null matches = 0
  | otherwise    = head matches
  where
    matches = [q | (r, q) <- xs, r == p]

checkStock :: Stock -> ShoppingCart -> [Product]
checkStock stock (ShoppingCart items) =
  [ p | CartItem p q <- items, stockQuantity stock p < q ]

### Testing

**cartOK**: Tests that a cart with quantities fully within stock correctly returns an empty list (no missing products).

**cartOver**: Tests that an item whose quantity exceeds stock is detected and returned as insufficient.

**cartMultiple**: Tests that when several items exceed stock, all of them are correctly reported.

**cartEmpty**: Tests that an empty cart produces no missing-stock warnings (returns an empty list).

In [23]:
product1 = Product 1 "Ipad" 12 Electronics
product2 = Product 2 "Blouse"  15 Clothing
product3 = Product 3 "Lemons"  5 Groceries

stock = Stock (zip [product1, product2, product3] [2, 3, 4])

cartOK = ShoppingCart [ CartItem product1 2, CartItem product2 1]
cartOver = ShoppingCart[ CartItem product1 2, CartItem product2 5]
cartMultiple = ShoppingCart[ CartItem product1 3, CartItem product2 2, CartItem product3 10]
cartEmpty = ShoppingCart []

--putStrLn $ listToString $ checkStock stock cartOK
checkStock stock cartOK
putStrLn $ listToString $ checkStock stock cartOver
putStrLn $ listToString $ checkStock stock cartMultiple
--putStrLn $ listToString $ checkStock stock cartEmpty
checkStock stock cartEmpty

[]

Product id: 2, product: "Blouse", price: 15.00, category: Clothing

Product id: 1, product: "Ipad", price: 12.00, category: Electronics
Product id: 3, product: "Lemons", price: 5.00, category: Groceries

[]

## 7. Order Creation

`createOrder` was designed to validate a cart before building an **Order**. It first checks which products are missing using `checkStock`. If the list is empty, stock is sufficient, so the function returns `Right` with a new order in the **Pending** state and its total price computed by `calculateOrderTotal`. If any product is missing, it returns `Left (Error missing)`. An order can only be created when all items are available.

In [24]:
createOrder :: Stock -> Customer -> ShoppingCart -> Either Error Order
createOrder stock customer c@(ShoppingCart xs)
    | null missing = Right (Order customer c totalPrice Pending)
    | otherwise    = Left (Error missing)
    where
        missing    = checkStock stock c
        totalPrice = calculateOrderTotal (Order customer c 0 Pending)

### Testing

In [25]:
product1 = Product 1 "Ipad" 12 Electronics
product2 = Product 2 "Blouse"  15 Clothing
product3 = Product 3 "Lemons"  5 Groceries

cart1 = CartItem product1 2
cart2 = CartItem product2 5
cart  = ShoppingCart [cart1, cart2]

stock = Stock (zip [product1, product2, product3] [2, 3, 4])

customerGold   = Customer 1 "Ivan"   Gold
customerSilver = Customer 2 "Isa"  Silver

In [26]:
cartOK = ShoppingCart [ CartItem product1 2, CartItem product2 1]
createOrder stock customerGold cartOK

Right Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.00, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.00, category: Clothing, units: 1
Total price: 35.10, status: Pending

In [27]:
cartOver = ShoppingCart [ CartItem product1 2, CartItem product2 5]
createOrder stock customerSilver cartOver

Left Error: The following products are out of stock:
	Product id: 2, product: "Blouse", price: 15.00, category: Clothing

In [28]:
cartMultiple = ShoppingCart[ CartItem product1 3, CartItem product2 2, CartItem product3 10]
createOrder stock customerGold cartMultiple

Left Error: The following products are out of stock:
	Product id: 1, product: "Ipad", price: 12.00, category: Electronics
	Product id: 3, product: "Lemons", price: 5.00, category: Groceries

In [29]:
cartOKAY = ShoppingCart [ CartItem product1 2, CartItem product2 3, CartItem product3 4]
createOrder stock customerSilver cartOKAY

Right Order for customer: Customer id: 2, customer: "Isa", loyalty level: Silver
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.00, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.00, category: Clothing, units: 3
	Item: Product id: 3, product: "Lemons", price: 5.00, category: Groceries, units: 4
Total price: 84.55, status: Pending

## 8. Status update

`transitionAllowed` defines all the valid movements between order **Status** values. Each allowed pair is listed explicitly, and any other combination returns `False`. We explicitly list the transicions since succ and pred wouldn't consider imposible transitions.

`updateOrderStatus` uses this rule to control state changes. It compares the old status of the order with the new one: if the transition is valid, it returns `Just` with the updated order; if not, it returns `Nothing`. This ensures that orders can only move forward through logical states and prevents impossible transitions.

In [30]:
transitionAllowed :: Status -> Status -> Bool
transitionAllowed Pending Processing = True
transitionAllowed Pending Cancelled  = True
transitionAllowed Processing Shipped    = True
transitionAllowed Processing Cancelled  = True
transitionAllowed Shipped Delivered  = True
transitionAllowed Shipped Cancelled  = True
transitionAllowed _ _ = False

updateOrderStatus :: Order -> Status -> Maybe Order
updateOrderStatus o@(Order _ _ _ old) new
  | transitionAllowed old new = Just (o { status = new })
  | otherwise             = Nothing

### Testing

In [31]:
product1 = Product 1 "Ipad" 12 Electronics
product2 = Product 2 "Blouse"  15 Clothing

cart  = ShoppingCart [CartItem product1 2, CartItem product2 1]
stock = Stock (zip [product1, product2] [2, 3])
customerGold = Customer 1 "Ivan" Gold

orderTest' = Order customer cart 0 Pending
orderTest = orderTest' { totalPrice = calculateOrderTotal orderTest' }
orderTest

Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.00, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.00, category: Clothing, units: 1
Total price: 35.10, status: Pending

Valid transitions

In [32]:
orderProc  = updateOrderStatus orderTest Processing
orderProc

Just Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.00, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.00, category: Clothing, units: 1
Total price: 35.10, status: Processing

In [33]:
orderShip  = orderProc  >>= (`updateOrderStatus` Shipped)
orderShip

Just Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.00, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.00, category: Clothing, units: 1
Total price: 35.10, status: Shipped

In [34]:
orderDelivered = orderShip  >>= (`updateOrderStatus` Delivered)
orderDelivered

Just Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.00, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.00, category: Clothing, units: 1
Total price: 35.10, status: Delivered

Invalid transitions

In [35]:
orderDelivered >>= (`updateOrderStatus` Pending)
orderDelivered >>= (`updateOrderStatus` Shipped)

Nothing

Nothing

## 9. Polymorphic Search:

`isProductByIdInCart` checks if the **Product** with provided `productId` exists in the **ShoppingCart** and return `True` in case of affirmation and `False` otherwise.

Following the same logic, `isProductByCategoryInCart` checks if at least there is one **Product** of provided `category` in the **ShoppingCart**.

`searchOrders` select all the **Order**s that satisfy all the criteria passed to function. To do so, it is recursively called on the orders that satisfy current (and previous if there are) criteria. To select only satisfying orders `filter` is used together with lambda functions. 

In [36]:
isProductByIdInCart :: Int -> ShoppingCart -> Bool
isProductByIdInCart _ (ShoppingCart []) = False
isProductByIdInCart productId (ShoppingCart ((CartItem (Product pid _ _ _) _):xs)) 
        | productId == pid = True
        | otherwise = isProductByIdInCart productId (ShoppingCart xs)


isProductByCategoryInCart :: Category -> ShoppingCart -> Bool
isProductByCategoryInCart _ (ShoppingCart []) = False
isProductByCategoryInCart category (ShoppingCart ((CartItem (Product _ _ _ c) _):xs)) 
        | category == c = True
        | otherwise = isProductByCategoryInCart category (ShoppingCart xs)



searchOrders :: [SearchCriterion] -> [Order] -> [Order]
searchOrders [] orders = orders
searchOrders ((ById id): xs) orders = searchOrders xs (filter (\(Order (Customer cid _ _) _ _ _) -> cid == id) orders)
searchOrders ((ByLoyaltyLevel loyaltyLevel): xs) orders = searchOrders xs (filter (\(Order (Customer _ _ ll) _ _ _) -> loyaltyLevel == ll) orders)
searchOrders ((ByProductId id): xs) orders = searchOrders xs (filter (\(Order _ cart _ _) -> isProductByIdInCart id cart) orders)
searchOrders ((ByCategory category): xs) orders = searchOrders xs (filter (\(Order _ cart _ _) -> isProductByCategoryInCart category cart) orders)
searchOrders ((ByTotalPrice price): xs) orders = searchOrders xs (filter (\(Order _ _ p _) -> price == p) orders)

### Testing

In [37]:
product1 = Product 1 "Ipad" 12 Electronics
product2 = Product 2 "Blouse"  15 Clothing
product3 = Product 3 "Lemons"  5 Groceries

item1 = CartItem product1 2
item2 = CartItem product2 5
item3 = CartItem product3 10
cart1 = ShoppingCart [item1,  item2]
cart2 = ShoppingCart [item2,  item3]


customer1 = Customer 1 "Ivan" Gold
customer2 = Customer 2 "Isa" Bronze
order1 = Order customer1 cart1 17 Pending
order2 = Order customer2 cart2 20 Processing
order3 = Order customer2 cart2 17 Shipped

In [38]:
searchOrders [ById 2] [order1, order2, order3]

[Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Blouse", price: 15.00, category: Clothing, units: 5
	Item: Product id: 3, product: "Lemons", price: 5.00, category: Groceries, units: 10
Total price: 20.00, status: Processing
,Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Blouse", price: 15.00, category: Clothing, units: 5
	Item: Product id: 3, product: "Lemons", price: 5.00, category: Groceries, units: 10
Total price: 17.00, status: Shipped
]

In [39]:
searchOrders [ByTotalPrice 17] [order1, order2, order3]

[Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.00, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.00, category: Clothing, units: 5
Total price: 17.00, status: Pending
,Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Blouse", price: 15.00, category: Clothing, units: 5
	Item: Product id: 3, product: "Lemons", price: 5.00, category: Groceries, units: 10
Total price: 17.00, status: Shipped
]

In [40]:
searchOrders [ByTotalPrice 17, ById 2] [order1, order2, order3]

[Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Blouse", price: 15.00, category: Clothing, units: 5
	Item: Product id: 3, product: "Lemons", price: 5.00, category: Groceries, units: 10
Total price: 17.00, status: Shipped
]

## 10. Customer Management

`isActive`determines whether an order is still active. It returns `False` for `Delivered` or `Cancelled`, and `True` for any other status. This helps filter out orders that should not be counted.

`activeOrders` takes a list of orders and keeps only those whose status is active. It uses `isActive` to remove delivered or cancelled orders.

`customersOf` extracts the customer of each order from a list of orders. It simply collects all customers without checking duplicates.

`uniq` removes repeated elements from a list while keeping the last occurrence. This ensures each customer appears once when computing totals.

`totalOfCustomer`computes how much a given customer has spent across a list of orders. It sums the `t` values of all orders whose customer matches the one given.

In [41]:
-- helper 1
isActive :: Status -> Bool
isActive Delivered = False
isActive Cancelled = False
isActive _         = True

-- helper 2
activeOrders :: [Order] -> [Order]
activeOrders os =
  [ o | o@(Order _ _ _ st) <- os, isActive st ] --call to helper1
-- helper 3
customersOf :: [Order] -> [Customer]
customersOf os =
  [ c | Order c _ _ _ <- os ]

-- helper4 (no duplicates)
uniq :: Eq a => [a] -> [a]
uniq [] = []
uniq (x:xs)
  | x `elem` xs = uniq xs
  | otherwise   = x : uniq xs

-- helper 5
totalOfCustomer :: Customer -> [Order] -> Float
totalOfCustomer cust os =
  sum [ t | Order c _ t _ <- os, c == cust ]

`highValueCustomers` returns all customers who have active orders whose total value exceeds a given limit. It filters orders with `activeOrders`, extracts unique customers, and checks each customer’s total using `totalOfCustomer`.

In [42]:
highValueCustomers :: [Order] -> Float -> [Customer]
highValueCustomers orders limit =
  [ c | c <- uniq (customersOf active), totalOfCustomer c active > limit] -- calls to helper 3 and helper 5
  where
    active = activeOrders orders --call ti helper 2

### Testing

We created a wide range of customers, products, carts, and orders to exercise every part of the logic. By combining different cart sizes, product prices, and order statuses, we produced both active and inactive orders with high, medium, and low totals. This allowed us to verify `customersOf`, `uniq`, `activeOrders`, `totalOfCustomer`, and finally `highValueCustomers` under many realistic scenarios. The variety of combinations ensures that the function is tested exhaustively and behaves correctly for multiple customers, mixed order statuses, and different spending levels.

In [43]:
customer1 = Customer 1 "Ivan" Gold
customer2 = Customer 2 "Isa" Gold
customer3 = Customer 3 "Siro" Bronze
customer4 = Customer 4 "Yan" Silver

product1 = Product 1 "Book" 12 Books
product2 = Product 2 "Lemons" 1.5 Groceries
product3 = Product 3 "Sandals" 50 Clothing
product4 = Product 4 "IPhone" 999 Electronics

item1 = CartItem product1 2
item2 = CartItem product1 1
item3 = CartItem product2 5
item4 = CartItem product2 2
item5 = CartItem product3 2
item6 = CartItem product3 1
item7 = CartItem product4 1
item8 = CartItem product4 2

cart1 = ShoppingCart [item1, item3]
cart2 = ShoppingCart [item2, item4]
cart3 = ShoppingCart [item5, item7]
cart4 = ShoppingCart [item6, item8]
cart5 = ShoppingCart [item1, item3, item5, item7]
cart6 = ShoppingCart [item2, item4, item6, item8]
cart7 = ShoppingCart [item1, item2, item3, item4, item5, item6, item7, item8]

order1' = Order customer1 cart1 0 Pending
order1 = order1' {totalPrice = calculateOrderTotal order1'}
order2' = Order customer1 cart2 0 Processing
order2 = order2' {totalPrice = calculateOrderTotal order2'}
order3' = Order customer1 cart3 0 Delivered
order3 = order3' {totalPrice = calculateOrderTotal order3'}
order4' = Order customer2 cart4 0 Shipped
order4 = order4' {totalPrice = calculateOrderTotal order4'}
order5' = Order customer3 cart5 0 Cancelled
order5 = order5' {totalPrice = calculateOrderTotal order5'}
order6' = Order customer3 cart6 0 Shipped
order6 = order6' {totalPrice = calculateOrderTotal order6'}
order7' = Order customer4 cart7 0 Cancelled
order7 = order7' {totalPrice = calculateOrderTotal order7'}

orders = [order1, order2, order3, order4, order5, order6, order7]
unique = uniq $ customersOf orders

active = activeOrders orders

putStrLn $ listToString $ highValueCustomers orders 1900
putStrLn $ listToString $ highValueCustomers orders 190
putStrLn $ listToString $ highValueCustomers orders 19

Customer id: 3, customer: "Siro", loyalty level: Bronze

Customer id: 2, customer: "Isa", loyalty level: Gold
Customer id: 3, customer: "Siro", loyalty level: Bronze

Customer id: 1, customer: "Ivan", loyalty level: Gold
Customer id: 2, customer: "Isa", loyalty level: Gold
Customer id: 3, customer: "Siro", loyalty level: Bronze

## File interaction

### Customer

Functions to save and read Customer to/from file

`formatCustomer` converts an **Customer** object into a comma-separated string suitable for saving to a file.

`saveCustomersToHandle` writes a list of **Customer**s to a file handle.

`saveCustomersToFile` writes a list of **Customer**s to a file with `filepath` given.

`parseCustomerLine` converts a comma-separated string into a **Customer** object.

`readCustomer` reads the entire content of a file and converts it into a list of **Customer** objects applying `parseCustomerLine` to every line.

In [44]:
formatCustomer :: Customer -> String
formatCustomer (Customer id name loyaltyLevel) = show id ++ "," ++ name ++ "," ++ show loyaltyLevel

saveCustomersToHandle :: Handle -> [Customer] -> IO ()
saveCustomersToHandle handle = mapM_ (hPutStrLn handle . formatCustomer)

saveCustomersToFile :: FilePath -> [Customer] -> IO ()
saveCustomersToFile filepath customer = 
    withFile filepath WriteMode $ \handle -> 
        saveCustomersToHandle handle customer

parseCustomerLine :: String -> Customer
parseCustomerLine line = Customer id name loyaltyLevel
  where
    [idStr, name, loyaltyLevelStr] = splitOn ',' line
    id = read idStr :: Int
    loyaltyLevel = read loyaltyLevelStr :: LoyaltyLevel

readCustomer :: FilePath -> IO [Customer]
readCustomer fp = do
  contents <- readFile fp
  forceCustomers (map parseCustomerLine (lines contents))
  where
    forceCustomers :: [Customer] -> IO [Customer]
    forceCustomers []     = return []
    forceCustomers (c:cs) = do
      c `seq` forceCustomers cs
      return (c:cs)

### Testing

In [45]:
main :: IO ()
main = do    
    saveCustomersToFile "test.txt" [Customer 1 "Alice" Gold, Customer 2 "Bob" Silver]
    putStrLn "Customers saved!"
    
    loadedCustomers <- readCustomer "test.txt"
    putStrLn "Customers loaded:"
    putStrLn $ listToString loadedCustomers
main

Customers saved!
Customers loaded:
Customer id: 1, customer: "Alice", loyalty level: Gold
Customer id: 2, customer: "Bob", loyalty level: Silver

### Stock

Functions to save and read Stock to/from file

`formatStock` converts an **Stock** object into a comma-separated string suitable for saving to a file.

`saveStockToHandle` writes a **Stock** to a file handle.

`saveStockToFile` writes a list of **Stock** to a file with `filepath` given.

`parseProductQuantityLine` converts a comma-separated string into a pair **(Product, Int)**

`readStock` reads the entire content of a file and converts it into a list of pairs **(Product, Int)** and then create **Stock** from that.

In [46]:
formatStock :: Stock -> String
formatStock (Stock items) = unlines [show id ++ "," ++ name ++ "," ++ show price ++ "," ++ show category ++ "," ++ show quantity | (Product id name price category, quantity) <- items]

saveStockToHandle :: Handle -> Stock -> IO ()
saveStockToHandle handle stock = hPutStrLn handle (formatStock stock)

saveStockToFile :: FilePath -> Stock -> IO ()
saveStockToFile filepath stock = 
    withFile filepath WriteMode $ \handle -> 
        saveStockToHandle handle stock

parseProductQuantityLine :: String -> (Product, Int)
parseProductQuantityLine line = (Product id name price category, qty)
  where
    [idStr, name, priceStr, categoryStr, qtyStr] = splitOn ',' line
    id = read idStr :: Int
    price = read priceStr :: Float
    category = read categoryStr :: Category
    qty = read qtyStr :: Int


readStock :: FilePath -> IO Stock
readStock fp = do
  contents <- readFile fp
  forceList (parseAll (lines contents))
  where
    parseAll :: [String] -> [(Product, Int)]
    parseAll [] = []
    parseAll (x:xs)
      | null x    = parseAll xs
      | otherwise = parseProductQuantityLine x : parseAll xs

    forceList :: [(Product, Int)] -> IO Stock
    forceList [] = return (Stock [])
    forceList ((p,q):rest) = do
      p `seq` q `seq` forceList rest
      return (Stock ((p,q):rest))

### Testing

In [47]:
main :: IO ()
main = do
    saveStockToFile "test.txt" (Stock 
        [ (Product 1 "Laptop" 999.99 Electronics, 5)
        , (Product 2 "Book" 19.99 Books, 10) 
        ])
    putStrLn "Stock saved!"
    
    loadedStock <- readStock "test.txt"
    putStrLn "Stock loaded:"
    print loadedStock
    
main

Stock saved!
Stock loaded:
Stock:
	Product id: 1, product: "Laptop", price: 999.99, category: Electronics, quantity available: 5
	Product id: 2, product: "Book", price: 19.99, category: Books, quantity available: 10

### Order

Functions to save Order to file

Order stores in a file in the following format:
- `Customer cid,cname,loyaltyLevel`  -- the first line, customer information
- `pid,pname,price,category,quantity *` -- as many lines as there are cart items
- `Price totalPrice,status` -- the last line, total price of order as well as its status

`formatCartItem` converts a **CartItem**, object into a comma-separated string suitable for saving to a file.

`formatPriceStatus` converts a `totalPrice` and **Status** of **Order**, intro specific string suitable for saving to a file.

`saveOrderToHandle` writes an **Order** to a file handle.

`saveOrdersToFile` writes a list of **Order** to a file with `filepath` given.



In [48]:
formatCartItem :: CartItem -> String
formatCartItem (CartItem (Product id name price category) qty) =
    show id ++ "," ++ name ++ "," ++ show price ++ "," ++ show category ++ "," ++ show qty

formatPriceStatus :: Float -> Status -> String
formatPriceStatus totalPrice status = 
    "Price " ++ show totalPrice ++ "," ++ show status

saveOrderToHandle :: Handle -> Order -> IO ()
saveOrderToHandle handle (Order cust (ShoppingCart items) totalPrice status) = do
    hPutStrLn handle ("Customer " ++ formatCustomer cust)
    mapM_ (hPutStrLn handle . formatCartItem) items
    hPutStrLn handle (formatPriceStatus totalPrice status)

saveOrdersToFile :: FilePath -> [Order] -> IO ()
saveOrdersToFile filepath orders = 
    withFile filepath WriteMode $ \handle ->
        mapM_ (saveOrderToHandle handle) orders

Functions to read Orders from file

`parsePriceStatusLine` converts a comma-separated string into a pair **(Float, Status)** removing "Price " prefix.

`readCartItems` reads all cart item lines until we hit a "Price" line returning pair `(**CartItem**s read, Strings left)`.

`readOrderFromLines` reads a single order from a list of lines returning a pair `(Order, remaining lines)`

`readAllOrdersFromLines` reads all **Order**s from a **String**s.

`readOrders` reads all **Order**s from a file specified by `filepath`

In [49]:
parsePriceStatusLine :: String -> (Float, Status)
parsePriceStatusLine line = (totalPrice, status)
  where
    [priceStr, statusStr] = splitOn ',' (drop 6 line)
    totalPrice = read priceStr :: Float
    status = read statusStr :: Status


readCartItems :: [String] -> ([CartItem], [String])
readCartItems [] = ([], [])
readCartItems (line:rest)
    | take 5 line == "Price" = ([], line:rest)
    | otherwise = (CartItem product qty : items, remaining)
  where
    (items, remaining) = readCartItems rest
    (product, qty) = parseProductQuantityLine line

  
readOrderFromLines :: [String] -> (Order, [String])
readOrderFromLines (customerLine:rest) = (order, remaining)
  where
    customer = parseCustomerLine (drop 9 customerLine) -- Remove "Customer " prefix
    (cartItems, priceAndRest) = readCartItems rest
    (totalPrice, status) = parsePriceStatusLine (head priceAndRest)
    shoppingCart = ShoppingCart cartItems
    order = Order customer shoppingCart totalPrice status
    remaining = tail priceAndRest


readAllOrdersFromLines :: [String] -> [Order]
readAllOrdersFromLines [] = []
readAllOrdersFromLines lines = order : readAllOrdersFromLines remaining
  where
    (order, remaining) = readOrderFromLines lines


readOrders :: FilePath -> IO [Order]
readOrders filepath = do
  contents <- readFile filepath
  forceList (readAllOrdersFromLines (lines contents))
  where -- strictly evaluate head, then recurse to force the entire list
    forceList :: [Order] -> IO [Order]
    forceList []     = return []
    forceList (o:os) = do
      o `seq` forceList os
      return (o:os)

### Testing

In [50]:
-- Main function to test
main :: IO ()
main = do
    saveOrdersToFile "test.txt" [Order (Customer 1 "Ivan" Gold) 
                                        (ShoppingCart 
                                            [CartItem (Product 1 "charger" 12.0 Electronics) 2, 
                                            CartItem (Product 2 "shoes" 15.0 Clothing) 5]) 
                                        89.1 Pending, 
                                    Order (Customer 2 "Maria" Silver) 
                                        (ShoppingCart 
                                            [CartItem (Product 3 "book" 20.0 Books) 1]) 
                                        19.0 Processing
                                    ]
    putStrLn "Orders saved!"
    
    
    loadedOrders <- readOrders "test.txt"
    putStrLn "Orders loaded:"
    mapM_ print loadedOrders

main

Orders saved!
Orders loaded:
Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "charger", price: 12.00, category: Electronics, units: 2
	Item: Product id: 2, product: "shoes", price: 15.00, category: Clothing, units: 5
Total price: 89.10, status: Pending

Order for customer: Customer id: 2, customer: "Maria", loyalty level: Silver
Shopping cart:
	Item: Product id: 3, product: "book", price: 20.00, category: Books, units: 1
Total price: 19.00, status: Processing

## 11. Customer interaction
Functions to use I/O to allow the user to interact with the system. He/she will be able to search by Product, by Category or by product maximum price, to add elements to the Cart and to place an Order.

`searchProductsByName` is a function designed to look for products with a specific name inside a list of products. It returns all products whose name matches exactly the one given by the user.

`searchProductsByCategory` was designedto help with the filtering of products by their category. If the user chooses a category such as *Books* or *Electronics*, it returns only the products that belong to that category.

`searchProductsByMaxPrice` helps with the search of all products whose price is less than or equal to a maximum value given by the user. It is useful when the user wants to see only affordable options.

`findProductById` looks for a product with a specific ID. If the product exists, it returns `Just product`; if not, it returns `Nothing`. This is important when adding items to a cart based on the product ID entered by the user.

`readCategory` converts a text input (like *"Books"* or *"Clothing"*) into the corresponding `Category` value. If the user writes something invalid, it returns `Nothing`.

`printProducts` prints a list of products to the screen. If the list is empty, it tells the user that no products were found. If there are products, it prints them one by one.

In [51]:
 -- each user needs to identify himself/herself --helper 1 - search by name
searchProductsByName :: String -> [Product] -> [Product]
searchProductsByName name ps =
  [ p | p@(Product _ n _ _) <- ps, n == name ]
  
--helper 2- search by category
searchProductsByCategory :: Category -> [Product] -> [Product]
searchProductsByCategory cat ps =
  [ p | p@(Product _ _ _ c) <- ps, c == cat ]

-- helper 3 - search by max price
searchProductsByMaxPrice :: Float -> [Product] -> [Product]
searchProductsByMaxPrice maxP ps =
  [ prod | prod@(Product _ _ price _) <- ps, price <= maxP ]
  
-- helper 4- prod by id
findProductById :: Int -> [Product] -> Maybe Product
findProductById _ [] = Nothing
findProductById pid (p@(Product pid' _ _ _) : ps)
  | pid == pid' = Just p
  | otherwise   = findProductById pid ps

-- helper 5 - read categ from user
readCategory :: String -> Maybe Category
readCategory "Electronics" = Just Electronics
readCategory "Books"       = Just Books
readCategory "Clothing"    = Just Clothing
readCategory "Groceries"   = Just Groceries
readCategory _             = Nothing

-- helper 6 - printer
printProducts :: [Product] -> IO ()
printProducts [] = putStrLn "No products found."
printProducts [p] = print p
printProducts (p:ps) = do
  print p
  printProducts ps

### Testing
`searchProductsByName`**`"Book"`** checks that the function correctly finds products whose name matches.

`searchProductsByCategory`**`Books`** verifies that filtering by category returns only products of that category.

`searchProductsByMaxPrice`**`20`** ensures the function keeps products priced at or below the limit.

`findProductById`**`3`** tests that an existing product ID is found and returned as `Just product`.

`findProductById`**`99`** confirms correct behaviour when the product is missing (`Nothing`).

`readCategory`**`"Electronics"`** checks that a valid category string is parsed correctly.

`readCategory`**`"Food"`** ensures invalid category input produces `Nothing`.

In [52]:
p1 = Product 1 "Ipad"   999 Electronics
p2 = Product 2 "Book"    15 Books
p3 = Product 3 "Blouse"  25 Clothing
p4 = Product 4 "Lemons"   3 Groceries

catalog = [p1, p2, p3, p4]

searchProductsByName "Book" catalog
searchProductsByCategory Books catalog
searchProductsByMaxPrice 20 catalog
findProductById 3 catalog
findProductById 99 catalog
test_readCategory1 = readCategory "Electronics"
test_readCategory2 = readCategory "Food"

[Product id: 2, product: "Book", price: 15.00, category: Books]

[Product id: 2, product: "Book", price: 15.00, category: Books]

[Product id: 2, product: "Book", price: 15.00, category: Books,Product id: 4, product: "Lemons", price: 3.00, category: Groceries]

Just Product id: 3, product: "Blouse", price: 25.00, category: Clothing

Nothing

#### Product search

`catalogFromStock` builds the list of available products by keeping only those with a positive quantity. It ensures that all later search operations work over a realistic catalog and relies on the data structure from the stock-loaded file.

`searchProductIO` allows the user to search by product name. It reads the user’s input, then applies the earlier helper `searchProductsByName` to filter the catalog created by `catalogFromStock`, and finally prints the results using `printProducts`.

`searchCategoryIO` asks the user for a category and first validates the input using the helper `readCategory`. If the category is valid, it applies the helper `searchProductsByCategory` on the catalog from `catalogFromStock` and prints the results. If the category is invalid, it shows an error message instead. This prevents invalid input from breaking the search.

`searchMaxPriceIO` lets the user search by price. It reads the input and uses `readMaybe` to check whether it is a valid number. If so, it applies the helper `searchProductsByMaxPrice` to the catalog from `catalogFromStock`. If not, it prints an informative error. This design makes price-based searching safe and user-friendly.

Together, these functions combine earlier pure search helpers with controlled user interaction, ensuring reliable behaviour even when users enter incorrect or unexpected input.

In [53]:
-- Product search I/O -- derive catalog from Stock
catalogFromStock :: Stock -> [Product]
catalogFromStock (Stock items) = [ p | (p, qty) <- items, qty > 0 ]

searchProductIO :: Stock -> IO ()
searchProductIO stock = do
  putStrLn "Enter product name:"
  name <- getLine
  printProducts (searchProductsByName name (catalogFromStock stock))

searchCategoryIO :: Stock -> IO ()
searchCategoryIO stock =
  do putStrLn "Enter category (Electronics, Books, Clothing, Groceries):"
     s <- getLine
     handle (readCategory s)
  where
    handle Nothing = putStrLn "Unknown category."
    handle (Just cat) = printProducts (searchProductsByCategory cat (catalogFromStock stock))

searchMaxPriceIO :: Stock -> IO ()
searchMaxPriceIO stock =
  do putStrLn "Enter maximum price:"
     s <- getLine
     handle (readMaybe s :: Maybe Float)
  where
    handle :: Maybe Float -> IO ()
    handle Nothing = putStrLn "That is not a number."
    handle (Just mp) =  printProducts (searchProductsByMaxPrice mp (catalogFromStock stock))

#### Cart handling

`addToCartIO` handles the interaction needed to add a product to the shopping cart. It asks the user for a product ID and a quantity, attempts to convert both using `readMaybe`, and checks that the inputs are valid. When the inputs are correct, it uses `addToCartHelper` together with the catalog obtained from `catalogFromStock` to continue the process. If either value is invalid, it prints an error and keeps the cart unchanged.

`addToCartHelper` receives a product ID, a quantity, and the catalog. It uses the earlier helper `findProductById` to search for the product inside the catalog. Instead of directly modifying the cart, it delegates the next step to `addToCartMaybe`, which handles the two possible outcomes.

`addToCartMaybe` checks whether the product was found. If `findProductById` returned `Nothing`, it tells the user that the product does not exist and returns the cart unchanged. If a product was found (`Just p`), it confirms the addition and uses the previously defined `addToCart` function to insert a `CartItem` into the cart.

Together, these functions combine user input handling, safe parsing, catalog lookup using helper functions, and controlled updates to the cart. They ensure that only valid products with valid quantities are added, while keeping the system robust against incorrect user input.

In [54]:
-- Add to cart I/O -- use addToCart from 5
addToCartIO :: Stock -> ShoppingCart -> IO ShoppingCart
addToCartIO stock cart =
  do putStrLn "Enter product ID:"
     sId  <- getLine
     putStrLn "Enter quantity:"
     sQty <- getLine
     processInputs (readMaybe sId :: Maybe Int) (readMaybe sQty :: Maybe Int)
  where
    processInputs :: Maybe Int -> Maybe Int -> IO ShoppingCart
    processInputs (Just pid) (Just qty) =
      addToCartHelper pid qty (catalogFromStock stock) cart
    processInputs _ _ =
      do putStrLn "Invalid ID or quantity."
         return cart

addToCartHelper :: Int -> Int -> [Product] -> ShoppingCart -> IO ShoppingCart
--addToCartHelper pid qty catalog cart = addToCartMaybe (findProductById pid catalog) qty cart
addToCartHelper pid qty catalog = addToCartMaybe (findProductById pid catalog) qty

addToCartMaybe :: Maybe Product -> Int -> ShoppingCart -> IO ShoppingCart
addToCartMaybe Nothing _ cart =
  do putStrLn "Product not found."
     return cart
addToCartMaybe (Just p) qty cart =
  do putStrLn "Product added to cart."
     return (addToCart (CartItem p qty) cart)

#### Customer handling

`findCustomerById` searches a list of customers by their numeric ID. It returns `Just customer` when it finds a match and `Nothing` otherwise. This helper lets us safely look up existing customers before starting any shopping actions.

`readLoyaltyLevel` converts a text input into a `LoyaltyLevel`. It first lowers the case of the input and then matches `"bronze"`, `"silver"`, or `"gold"`. Any other text returns `Nothing`. This design allows flexible user input (case-insensitive) while keeping the internal loyalty type strict and well-defined.

`nextCustomerId` computes the next available customer ID. If there are no customers, it starts at 1; otherwise, it takes the maximum existing ID and adds 1. This guarantees that each new customer gets a unique identifier without manual tracking.

`identifyCustomer` manages the main identification flow. It asks the user whether they are an existing customer or a new one, and then calls `existingCustomerFlow` or `newCustomerFlow` accordingly. It also handles the exit option (`"b"`) and invalid answers, so the system does not crash on wrong input.

`existingCustomerFlow` implements the logic for returning customers. It asks for the customer ID, tries to read it as an integer using `readMaybe`, and then uses `findCustomerById` to look up the customer. If the ID is not an integer or does not exist, it prints an error and repeats the process. If the customer is found, it greets them by name and returns the customer together with the unchanged list of customers.

`newCustomerFlow` is the entry point for new users. We decided to allow new customers to register directly in the system, without any file editing. It asks for the user’s name and then calls `success` with `Bronze` as the loyalty level, because a new customer has not yet shown any loyalty and should start at the lowest level by default.

`success` finishes the registration of a new customer. It computes a fresh ID using `nextCustomerId`, builds a new `Customer` with the given name and loyalty level, prints the assigned ID, and returns the new customer together with the updated customer list (original list plus the new entry). This neatly encapsulates the creation and storage of new customers in one place.

In [55]:
-- Customer identification (new vs existing)
findCustomerById :: Int -> [Customer] -> Maybe Customer
findCustomerById _ [] = Nothing
findCustomerById cid (c@(Customer cid' _ _) : cs)
  | cid == cid' = Just c
  | otherwise   = findCustomerById cid cs

readLoyaltyLevel :: String -> Maybe LoyaltyLevel
readLoyaltyLevel s
  | ls == "bronze" = Just Bronze
  | ls == "silver" = Just Silver
  | ls == "gold"   = Just Gold
  | otherwise      = Nothing
  where
    ls = map toLower s

nextCustomerId :: [Customer] -> Int
nextCustomerId [] = 1
nextCustomerId cs = 1 + maximum [cid | Customer cid _ _ <- cs]

identifyCustomer :: [Customer] -> IO (Customer, [Customer])
identifyCustomer customers = do
  putStrLn "Are you an existing customer (1) or a new customer (2)? (b to exit)"
  ans <- getLine
  handleAnswer ans customers
  where
    handleAnswer :: String -> [Customer] -> IO (Customer, [Customer])
    handleAnswer "1" cs = existingCustomerFlow cs
    handleAnswer "2" cs = newCustomerFlow cs
    handleAnswer "b" cs = do
      putStrLn "Goodbye."
      fail "User aborted"
    handleAnswer _ cs = do
      putStrLn "Unknown option"
      identifyCustomer cs

existingCustomerFlow :: [Customer] -> IO (Customer, [Customer])
existingCustomerFlow customers = do
  putStrLn "Please enter your customer ID:"
  sId <- getLine
  handleIdParse (readMaybe sId :: Maybe Int) customers
  where
    handleIdParse :: Maybe Int -> [Customer] -> IO (Customer, [Customer])
    handleIdParse Nothing cs = do
      putStrLn "ID must be an integer."
      existingCustomerFlow cs
    handleIdParse (Just cid) cs =
      handleCustomerLookup (findCustomerById cid cs) cs

    handleCustomerLookup :: Maybe Customer -> [Customer] -> IO (Customer, [Customer])
    handleCustomerLookup Nothing cs = do
      putStrLn "No customer with that ID."
      existingCustomerFlow cs
    handleCustomerLookup (Just c) cs = do
      putStrLn ("Welcome back, " ++ cname c ++ "!")
      return (c, cs)

newCustomerFlow :: [Customer] -> IO (Customer, [Customer])
newCustomerFlow customers = do
  putStrLn "Registering a new customer."
  putStrLn "Enter your name:"
  name <- getLine
  success customers name Bronze

success :: [Customer] -> String -> LoyaltyLevel -> IO (Customer, [Customer])
success customers name ll = do
  putStrLn ("Your new customer ID is " ++ show cid)
  return (newC, customers ++ [newC])
  where
    cid  = nextCustomerId customers
    newC = Customer cid name ll

Stock update after succesful order

In [56]:
updateStockWithCart :: Stock -> ShoppingCart -> Stock
updateStockWithCart (Stock items) (ShoppingCart cartItems) =
  Stock (foldl updateOne items cartItems)
  where
    updateOne :: [(Product, Int)] -> CartItem -> [(Product, Int)]
    updateOne acc (CartItem (Product pid _ _ _) q) =
      [ if pid' == pid then (p, qty - q) else (p, qty) | (p@(Product pid' _ _ _), qty) <- acc ]

#### Shopping loop

`shopLoop` manages the main interactive menu of the shop. It repeatedly shows the available actions and reads the user’s choice, making the program behave like a simple command-line interface. Each option calls one of the previously defined IO functions: searching (`searchProductIO`, `searchCategoryIO`, `searchMaxPriceIO`), adding items (`addToCartIO`), or viewing the cart. After performing the chosen action, the loop calls itself again to continue the session, except when the user selects checkout or exit, in which case it returns the final cart and stock. This design ties together all earlier components into a single continuous shopping flow.

In [57]:
shopLoop :: Stock -> ShoppingCart -> IO (ShoppingCart, Stock)
shopLoop stock cart = do
  putStrLn "\nWhat would you like to do?"
  putStrLn "1. Search products by name"
  putStrLn "2. Search products by category"
  putStrLn "3. Search products by maximum price"
  putStrLn "4. Add product to cart"
  putStrLn "5. View cart"
  putStrLn "6. Checkout"
  putStrLn "b. Exit without ordering"
  choice <- getLine
  handle choice stock cart
  where
    handle :: String -> Stock -> ShoppingCart -> IO (ShoppingCart, Stock)
    handle "1" st ct = do
      searchProductIO st
      shopLoop st ct
    handle "2" st ct = do
      searchCategoryIO st
      shopLoop st ct
    handle "3" st ct = do
      searchMaxPriceIO st
      shopLoop st ct
    handle "4" st ct = do
      ct' <- addToCartIO st ct
      shopLoop st ct'
    handle "5" st ct = do
      putStrLn "Your cart:"
      print ct
      shopLoop st ct
    handle "6" st ct =
      return (ct, st)
    handle "b" st ct =
      return (ct, st)
    handle _ st ct = do
      putStrLn "Unknown option."
      shopLoop st ct

### MAIN FUNCTIONS

`placeOrderIO` connects the pure order creation logic with the user interface. It calls `createOrder stock cust cart` and then inspects the result: if it returns `Left err`, it prints an error message and signals that the order could not be created; if it returns `Right order`, it shows the order on screen and returns it. This function is the bridge between `createOrder` (from step 7) and the rest of the IO flow.

`customerSession` coordinates a full session for one customer, from welcome to file saving. It first loads the current state of the system using `readCustomer`, `readStock`, and `readOrders`. Then it identifies the customer with `identifyCustomer` and starts the shopping loop using `shopLoop` on an empty cart. When the loop ends, `cartHandler` decides what to do: if the cart is empty, it only saves updated customers and stock and exits; if the cart has items, it tries to place an order using `placeOrderIO`. In case of failure, `orderResult` reports that the order failed and saves only customer and stock data as they were before the attempt. In case of success, it calls `updateStockWithCart` to reduce stock, appends the new order to the existing list, and then uses `saveCustomersToFile`, `saveStockToFile`, and `saveOrdersToFile` to persist all changes. Finally, `main` simply runs `customerSession`, making this complete flow the entry point of the program.

In [58]:
-- Place order using createOrder (from 7) and update files
placeOrderIO :: Stock -> Customer -> ShoppingCart -> IO (Either Error Order)
placeOrderIO stock cust cart =
  handleCreate (createOrder stock cust cart)
  where
    handleCreate (Left err) = do
      putStrLn ("Order could not be created: " ++ show err)
      return (Left err)
    handleCreate (Right order) = do
      putStrLn "Order created successfully:"
      print order
      return (Right order)

customerSession :: IO ()
customerSession = do
  putStrLn "Welcome to Ivan's and Isa's shop!"

  customers <- readCustomer "customers.txt"
  stock     <- readStock    "stock.txt"
  orders    <- readOrders "orders.txt"

  (customer, customers') <- identifyCustomer customers

  cartHandler (shopLoop stock (ShoppingCart [])) customer customers' stock orders
  where
    cartHandler action cust customers' st orders = do
      (finalCart, stockBeforeOrder) <- action
      handleCart finalCart cust customers' stockBeforeOrder orders

    handleCart (ShoppingCart []) _ customers' stockBeforeOrder orders = do
      putStrLn "Your cart is empty. Nothing to do."
      saveCustomersToFile "customers.txt" customers'
      saveStockToFile     "stock.txt"     stockBeforeOrder
      saveOrdersToFile    "orders.txt"    orders
      putStrLn "Goodbye."

    handleCart cart cust customers' stockBeforeOrder orders = do
      handleOrder (placeOrderIO stockBeforeOrder cust cart)
                  cart cust customers' stockBeforeOrder orders

    handleOrder action cart cust customers' stockBeforeOrder orders = do
      result <- action
      orderResult result cart cust customers' stockBeforeOrder orders

    orderResult (Left _) _ _ customers' stockBeforeOrder orders = do
      putStrLn "Order failed. No changes saved (except new customers)."
      saveCustomersToFile "customers.txt" customers'
      saveStockToFile     "stock.txt"     stockBeforeOrder
      saveOrdersToFile    "orders.txt"    orders

    orderResult (Right newOrder) cart _ customers' stockBeforeOrder orders = do
      putStrLn "Updating files..."
      saveCustomersToFile "customers.txt" customers'
      saveStockToFile     "stock.txt"     (updateStockWithCart stockBeforeOrder cart)
      saveOrdersToFile    "orders.txt"    (orders ++ [newOrder])
      putStrLn "Order placed and data saved. Thank you!"

main :: IO ()
main = customerSession

# 12. Manager Interaction
Functions to use I/O to manage the shop. The shop owner will be able to search by user ID, by Loyalty Level or by high value customers, and to process and ship all the pending orders

Update status of orders of `orders` that are in `toUpdate` 

In [59]:
updateOrders :: [Order] -> [Order] -> Status -> [Order] --F1
updateOrders toUpdate orders newStatus = map update orders
    where
        update :: Order -> Order
        update order
            | order `elem` toUpdate = fromMaybe order (updateOrderStatus order newStatus)
            | otherwise             = order

Double recursion approaches

In [60]:
-------------------------------------------------------------------------------------------------------------------------
-- Process and ship all the pending orders of customer with id given
-------------------------------------------------------------------------------------------------------------------------

ordersById :: Int -> [Order] -> IO [Order] --F2
ordersById id currentAllOrders
        | null userOrders = do
                putStrLn "No orders by user"
                searchById currentAllOrders -- let user search again
        | otherwise = do
                        newAllOrders <- return $ updateOrders userOrders currentAllOrders Processing
                        newAllOrders <- return $ updateOrders userOrders newAllOrders Shipped
                        putStrLn $ "Customer " ++ show id ++ " orders updated"
                        start newAllOrders -- go back to start with updated orders
        where userOrders = searchOrders [ById id] currentAllOrders

-------------------------------------------------------------------------------------------------------------------------
-- Process and ship all the pending orders of customers with loyalty level ll provided
-------------------------------------------------------------------------------------------------------------------------
        
ordersByLL :: LoyaltyLevel -> [Order] -> IO [Order] --F3
ordersByLL ll currentAllOrders
        | null usersOrders = do
                putStrLn "No orders by this Loyalty Level"
                searchByLL currentAllOrders
        | otherwise = do
                        newAllOrders <- return $ updateOrders usersOrders currentAllOrders Processing
                        newAllOrders <- return $ updateOrders usersOrders newAllOrders Shipped
                        putStrLn "Orders updated"
                        start newAllOrders
        where usersOrders = searchOrders [ByLoyaltyLevel ll] currentAllOrders
        
-------------------------------------------------------------------------------------------------------------------------
-- Process and ship all the pending orders of high value customers with limit total price of all active orders
-------------------------------------------------------------------------------------------------------------------------

ordersByHighValue :: [Order] -> [Customer] -> [Order] --F4
ordersByHighValue currentAllOrders customers = updateOrders customersOrders pendingToProcessing Shipped
        where   customersOrders = concat [ searchOrders [ById id] currentAllOrders | (Customer id _ _) <- customers ]
                pendingToProcessing = updateOrders customersOrders currentAllOrders Processing

-------------------------------------------------------------------------------------------------------------------------
-- Check user id provided for being correct and process further
-------------------------------------------------------------------------------------------------------------------------                
                
checkIdInput :: String -> [Order] -> IO [Order] --F5
checkIdInput id currentAllOrders
        -- | all isDigit id = ordersById (read id :: Int) currentAllOrders
        | id == "b" || id == "B" = startWork currentAllOrders
        | otherwise = do
                putStrLn "ID consist only of digits!"
                searchById currentAllOrders


searchById :: [Order] -> IO [Order] --F6
searchById currentAllOrders = do
        putStrLn "Introduce user ID (b to come back):"
        id <- getLine
        checkIdInput id currentAllOrders
        
-------------------------------------------------------------------------------------------------------------------------
-- Check loyalty level ll provided for being correct and process further
-------------------------------------------------------------------------------------------------------------------------         

searchByLLInputCheck :: String -> [Order] -> IO [Order] --F7
searchByLLInputCheck ll currentAllOrders
        | ll == "bronze" = ordersByLL Bronze currentAllOrders
        | ll == "silver" = ordersByLL Silver currentAllOrders
        | ll == "gold"   = ordersByLL Gold currentAllOrders
        | ll == "b"      = startWork currentAllOrders
        | otherwise = do
                putStrLn "There is no such Loyalty Level"
                searchByLL currentAllOrders


searchByLL :: [Order] -> IO [Order] --F8
searchByLL currentAllOrders = do
        putStrLn "Introduce user Loyalty Level (Bronze, Silver or Gold. b to come back): "
        ll <- getLine
        searchByLLInputCheck (map toLower ll) currentAllOrders
        
-------------------------------------------------------------------------------------------------------------------------
-- Checks limit provided for being proper number and process further
-------------------------------------------------------------------------------------------------------------------------         

usersByHighValue :: Float -> [Order] -> IO [Order] --F9
usersByHighValue limit currentAllOrders
        | limit > 0 = do
                putStrLn "Orders updated"
                start $ ordersByHighValue currentAllOrders (highValueCustomers currentAllOrders limit)
        | otherwise = do
                putStrLn "Total price should be non negative number"
                searchByHighValue currentAllOrders
                

searchByHighValueInputCheck :: String -> [Order] -> IO [Order] --F10
searchByHighValueInputCheck value currentAllOrders
        | all isDigit value = usersByHighValue (read value :: Float) currentAllOrders
        | value == "b" || value == "B" = startWork currentAllOrders
        | otherwise = do
                putStrLn "That is not a number"
                searchByHighValue currentAllOrders


searchByHighValue :: [Order] -> IO [Order] --F11
searchByHighValue currentAllOrders = do
        putStrLn "Introduce the minimum total price of not Delivered or Cancelled orders (b to come back):"
        value <- getLine
        searchByHighValueInputCheck value currentAllOrders
        
-------------------------------------------------------------------------------------------------------------------------
-- Offers options to select orders
-------------------------------------------------------------------------------------------------------------------------         

startWorkInputCheck :: String -> [Order] -> IO [Order] --F12
startWorkInputCheck answer currentAllOrders
        | answer == "1" = searchById currentAllOrders
        | answer == "2" = searchByLL currentAllOrders
        | answer == "3" = searchByHighValue currentAllOrders
        | answer == "b" = start currentAllOrders
        | otherwise = do
                putStrLn "There is no such option"
                startWork currentAllOrders

startWork :: [Order] -> IO [Order] --F13
startWork currentAllOrders = do 
        putStrLn "What do you want to do?"
        putStrLn "Options:\n1. Search orders by user ID\n2. Search orders by Loyalty Level"
        putStrLn "3. Search orders by high value customers"
        putStrLn "b. Back to main menu"
        answer <- getLine
        startWorkInputCheck answer currentAllOrders
        
-------------------------------------------------------------------------------------------------------------------------
-- Handles start input
-------------------------------------------------------------------------------------------------------------------------         

checkStartAnswer :: String -> [Order] -> IO [Order] --F14
checkStartAnswer answer currentAllOrders
        | answer `elem` ["yes", "yeah", "y"] = startWork currentAllOrders
        | answer `elem` ["no", "nope", "n", "nah"] = do
                putStrLn "Understood :("
                return currentAllOrders
        | otherwise = do
                    putStrLn "I do not understand you"
                    start currentAllOrders


start :: [Order] -> IO [Order] --F15
start currentAllOrders = do
        putStrLn "Do you want to continue?"
        answer <- getLine
        checkStartAnswer (map toLower answer) currentAllOrders

Starts owner session, load orders and later save them back to file

In [61]:
ownerSession :: IO ()
ownerSession = do
        putStrLn "Welcome back manager!"
        putStrLn "\nReading orders..."
        --loadedOrders <- readOrdersFromFile "orders.txt"
        loadedOrders <- readOrders "orders.txt"
        putStrLn "Orders loaded:"

        allorders <- start loadedOrders
        putStrLn "Bye, have a good day!"
        -- Save orders
        putStrLn "Saving orders..."
        saveOrdersToFile "orders.txt" allorders
        putStrLn "Orders saved!"

Start the whole session, allows to start it as owner, if password is know, or as a customer

In [62]:
{- 
checkPassword :: String -> Bool
checkPassword inputPass
        | inputPass == "IvanAndIsaAreTheBestShopOwners" = do 
                    putStrLn "Access granted."
                    ownerSession
        | otherwise = do
                    putStrLn "Incorrect password. Access denied."
                    session
-}

checkPassword :: String -> IO ()
checkPassword inputPass
  | inputPass == "1234" = do
      putStrLn "Access granted."
      ownerSession
  | otherwise = do
      putStrLn "Incorrect password. Access denied."
      session

session :: IO ()
session = do
        putStrLn "Are you a customer (c) or the shop owner (o)? (b to exit)"
        answer <- getLine
        handleAnswer answer
    where
        handleAnswer :: String -> IO ()
        handleAnswer "c" = customerSession
        handleAnswer "o" = do
                putStrLn "Please enter the owner password:"
                inputPass <- getLine
                checkPassword inputPass
        handleAnswer "b" = putStrLn "Goodbye."
        handleAnswer _   = do
                            putStrLn "Unknown option"
                            session

# Conclusion

The project was fun combined with struggle, but we managed to complete the required tasks and additionally got creative, decided it could be more user friendly, and managed to elevate the project to everything we wanted to achieve, such as new customer registration or identifying oneself as either customer or owner. We believe that this project helped us a lot to understand better Haskell as a language but additionally its application, because personally me (Ivan) did not believe that something comprehensive like this can be done using functional programming. 

It was interesting to explore and use Haskell functions that do not appear in other languages and to master recursion as well as exhaustive pattern matching (specifically in the order's status transitions). Additionally it was involving to study recommendations provided by Jupyter, as some of them allowed much simpler and more "beautiful" solutions.

Some of the problems we faced were in general idea how to do some functionality without classes and their methods (like the ones in OOP) and especially without variables and cycles (in common understanding), as we got used to those paradigms of programming and now we had to adapt. A clear key lesson was learning to avoid imperative habits. Refactoring the stock logic for example taught me (Isa) something similar. The preliminary version relied on a global stock variable, which made the program dependent making it awkward and conceptually incorrect. Rewriting `checkStock` to depend only on its explicit arguments removed hidden state and resulted in a cleaner and more general design.

Personally me (Ivan), did not like that some functionality is limited (due to my lack of knowledge or limitations of Haskell in comparison to other languages), like for examples limitation of constructor, we cannot directly while creation an "object", check values of parameters, like ids or quantities, as theoretically they can be set to negative values, so this is something I did not like, although we can use auxiliary functions to control that, still it is something that did not leave me calm.

The I/O system was the most demanding part. Handling user interaction, typed input, file updates, and error paths required many trial-and-error compiles. We also relied heavily on double recursion to manage the interaction loops, especially for the manager side of the system. Double recursion turned out to be one of the most powerful tools in the project: it allowed us to structure repeated choices, return paths, and nested flows in a clean and purely functional way. However, this strength also exposed a practical limitation. Jupyter executes Haskell code incrementally, cell by cell, and because our recursive functions called each other in both directions, Jupyter could not resolve them unless they all appeared together. The only way to make the system runnable in that environment was to place all mutually recursive I/O functions into a single cell. This was a clear example of how functional programming encourages elegant recursive designs, while Jupyter’s execution model is not aligned with that style. Moving the project to .hs files solved the issue and showed us how Haskell I/O programs are actually meant to be executed.

Overall, the main challenges were conceptual: modelling state without variables, working with pure functions, and understanding why constructors cannot enforce validation as in OOP. Even though auxiliary functions solve these issues, the adjustment required time.

For me (Isa), it was a privilege to have Ivan as my peer for this project. I truly learned a lot, and it is rare to reach such a strong level of understanding and coordination, specially in university projects. I believe the success of this project came from our clear communication and the genuine investment we both put into the work. Neither of us tried to overshadow the other; instead, we approached every task with real teamwork. We made sure that each of us understood the other’s individual implementations, which allowed us to build such a robust and coherent system.